In [2]:
import random
from typing import List, Tuple

import gym
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from IPython.display import clear_output
from torch.distributions import Normal
from unityagents import UnityEnvironment
import numpy as np
from reacher.src.networks import * 
from reacher.src.utils import *

In [3]:
device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu"
        )
print(device)

cuda


In [5]:
file_name = './Reacher_Windows_x86_One/Reacher_Windows_x86_64/Reacher.exe'
worker_id = 10
base_port = 5006
env, brain_name, brain, action_size, env_info, state, state_size, n_agents = load_env(worker_id,
                                                                                                  base_port, file_name,
                                                                                                  True, True)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [18]:
is_test = False
transition= []
gamma = 0.9
entropy_weight = 1e-2
TAU = 1e-3  # for soft update of target parameters
LR_ACTOR = 1e-4  # learning rate of the actor
LR_CRITIC = 1e-4  # learning rate of the critic
vmax = 10
vmin = -10
n_atoms = 51
N_step = 1
UPDATE_EVERY = 10
BATCH_SIZE = 64
num_episodes= 10000 

In [7]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

In [8]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -6.30408478e+00 -1.00000000e+00
 -4.92529202e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -5.33014059e-01]


In [10]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents) 
states,num_agents

(array([[ 0.00000000e+00, -4.00000000e+00,  0.00000000e+00,
          1.00000000e+00, -0.00000000e+00, -0.00000000e+00,
         -4.37113883e-08,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00, -1.00000000e+01,
          0.00000000e+00,  1.00000000e+00, -0.00000000e+00,
         -0.00000000e+00, -4.37113883e-08,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  7.90150642e+00,
         -1.00000000e+00,  1.25147498e+00,  0.00000000e+00,
          1.00000000e+00,  0.00000000e+00, -2.99753308e-01]]),
 1)

In [11]:
states

array([[ 0.00000000e+00, -4.00000000e+00,  0.00000000e+00,
         1.00000000e+00, -0.00000000e+00, -0.00000000e+00,
        -4.37113883e-08,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -1.00000000e+01,
         0.00000000e+00,  1.00000000e+00, -0.00000000e+00,
        -0.00000000e+00, -4.37113883e-08,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  7.90150642e+00,
        -1.00000000e+00,  1.25147498e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00, -2.99753308e-01]])

In [12]:
state_size, action_size

(33, 4)

In [13]:
obs_dim = state_size
action_dim = action_size
obs_dim, action_dim

(33, 4)

In [19]:
random_seed=42
# Actor Network (w/ Target Network)
actor_local = Actor_D4PG(state_size, action_size, random_seed).to(device)
actor_target = Actor_D4PG(state_size, action_size, random_seed).to(device)
actor_optimizer = optim.Adam(actor_local.parameters(), lr=LR_ACTOR)

# initialize with its own Learning Rate
actor_optimizer = optim.Adam(actor_local.parameters(), lr=LR_ACTOR)

# Critic Network
critic_local = CriticD4PG(state_size, action_size, random_seed, n_atoms=n_atoms, v_min=vmin,
                               v_max=vmax).to(device)
critic_target = CriticD4PG(state_size, action_size, random_seed, n_atoms=n_atoms, v_min=vmin,
                                v_max=vmax).to(device)
# initialize with its own Learning Rate
critic_optimizer = optim.Adam(critic_local.parameters(), lr=LR_CRITIC)

In [31]:
BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 128  # minibatch size
memory = ReplayBuffer(device, action_size, BUFFER_SIZE, BATCH_SIZE, random_seed)

In [20]:
add_noise=True
noise = OUNoise(action_size, random_seed)

In [21]:
env_info = env.reset(train_mode=True)[brain_name]  # reset the environment
states = env_info.vector_observations

score = np.zeros(n_agents)

In [22]:
states

array([[ 0.00000000e+00, -4.00000000e+00,  0.00000000e+00,
         1.00000000e+00, -0.00000000e+00, -0.00000000e+00,
        -4.37113883e-08,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -1.00000000e+01,
         0.00000000e+00,  1.00000000e+00, -0.00000000e+00,
        -0.00000000e+00, -4.37113883e-08,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  3.50696945e+00,
        -1.00000000e+00, -7.19035244e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00, -3.66842747e-03]])

In [24]:
state=torch.tensor(states).float().to(device)

In [25]:
actor_local.eval()
with torch.no_grad():
    action = actor_local(state).cpu().data.numpy()
actor_local.train()

Actor_D4PG(
  (fc1): Linear(in_features=33, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=4, bias=True)
  (tanh): Tanh()
)

In [26]:
action += noise.sample()
action = np.clip(action, -1.0, 1.0)
action

array([[-0.37913406,  0.13970536,  0.31412667,  0.17244571]],
      dtype=float32)

In [27]:
env_info = env.step(action)[brain_name]  # send the action to the environment
next_states = env_info.vector_observations  # get the next state
rewards = env_info.rewards  # get the reward
dones = env_info.local_done
next_states, rewards,dones

(array([[-7.78579712e-03, -3.99947238e+00,  7.05741048e-02,
          9.99961019e-01, -9.68279841e-04,  7.15464284e-06,
          8.77801143e-03, -3.49774599e-01, -4.06605803e-04,
         -3.84921357e-02, -1.54749066e-01,  1.50330188e-02,
          1.40603244e+00,  7.12203979e-03, -9.99792004e+00,
          3.00594456e-02,  9.99873459e-01,  3.45206005e-03,
          8.42855370e-05, -1.55305266e-02,  6.18893564e-01,
         -4.83668083e-03,  1.37593284e-01,  3.43007952e-01,
          8.03209916e-02, -3.01152080e-01,  3.50880623e+00,
         -1.00000000e+00, -7.18945599e+00,  0.00000000e+00,
          1.00000000e+00,  0.00000000e+00, -3.66842747e-03]]),
 [0.0],
 [False])

In [32]:
memory.add(states, action, rewards, next_states, dones)

In [36]:
memory.__len__()

1

In [40]:
#critic_local(states, action)

In [ ]:
actor = Actor(obs_dim, action_dim).to(device)
critic = Critic(obs_dim).to(device)

In [ ]:
actor_optimizer = optim.Adam(actor.parameters(), lr=1e-4)
critic_optimizer = optim.Adam(critic.parameters(), lr=1e-3)

In [ ]:
state = torch.from_numpy(states).float().to(device)
state

In [ ]:
action, dist = actor(state)
action, dist

In [ ]:
selected_action = dist.mean if is_test else action
selected_action

In [ ]:
if not is_test:
    log_prob = dist.log_prob(selected_action).sum(dim=-1)
    transition = [state, log_prob]
transition

In [ ]:
selected_action = selected_action.clamp(-2.0, 2.0).cpu().detach().numpy()
selected_action

In [ ]:
next_state, reward, done, _ = env.step(selected_action)
next_state, reward, done

In [ ]:
if not is_test:
    transition.extend([next_state, reward, done])  
transition

In [ ]:
state, log_prob, next_state, reward, done = transition
state, log_prob, next_state, reward, done

In [ ]:
mask = 1 - done
mask

In [ ]:
next_state = torch.FloatTensor(next_state).to(device)
next_state

In [ ]:
pred_value = critic(state)

In [ ]:
pred_value

In [ ]:
targ_value = reward + gamma * critic(next_state) * mask
targ_value

In [ ]:
value_loss = F.smooth_l1_loss(pred_value, targ_value.detach())
value_loss

In [ ]:
# update value
critic_optimizer.zero_grad()
value_loss.backward()
critic_optimizer.step()

In [ ]:
# advantage = Q_t - V(s_t)
advantage = (targ_value - pred_value).detach()
advantage

In [ ]:
policy_loss = -advantage * log_prob
policy_loss

In [ ]:
policy_loss += entropy_weight * -log_prob
policy_loss

In [ ]:
# update policy
actor_optimizer.zero_grad()
policy_loss.backward()
actor_optimizer.step()

In [ ]:
policy_loss.item(), value_loss.item()